# Limpieza

Aqui vamos a ir apuntando las cosas a limpiar y transformar:
- **Nombre de las columnas** -> Cambiar ('.'), por ('\_'), y quitar ('\_') si está a final de palabra.
- **age** -> Tiene nulos. Los gestionaria sustituyéndolos por la mediana. Cambiar de tipo float64 a tipo int64.
- **job** -> Quitar el ('.') a final de palabra. Tiene nulos.
- **marital** -> Pasar a minúsculas. Tiene nulos.
- **education** -> Cambiar ('.'), por ('_'). Tiene nulos.
- **default** -> Cambiar a str. Tiene nulos.
- **housing** -> Cambiar a str. Tiene nulos.
- **loan** -> Cambiar a str. Tiene nulos.
- **poutcome** -> Pasar a minúsculas.
- **cons.price.idx** ->  Cambiar (',') por ('.'). Tiene nulos.
- **cons.conf.idx** -> Cambiar (',') por ('.').
- **euribor3m** -> Cambiar (',') por ('.'). Tiene nulos.
- **nr.employed** -> Cambiar (',') por ('.').
- **y** -> Es un booleano. Cambiar a tipo bool.
- **date** -> El mes está en español. Cambiar al inglés y cambiarlo a tipo fecha. Tiene nulos.
- **contact_month** -> Crear columna nueva en base a date.
- **contact_year** -> Crear columna nueva en base a date.
- **latitude** -> No sabemos qué información nos aporta. La eliminaría.
- **longitude** -> No sabemos qué información nos aporta. La eliminaría.



In [1]:
#Tratamiento de datos
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#Exploracion de archivos
import sys
sys.path.append('../')

#Funciones personalizadas
from src import soporte_limpieza as spl 

In [2]:
df_o = pd.read_csv("../data/raw/bank-additional.csv", index_col=0)
df = df_o.copy()

### Nombre Columnas
Cambiar ('.') por ('\_'), eliminar ('\_') si está a final de palabra.

In [3]:
df.columns = df.columns.str.replace('.','_', regex=False).str.replace('_$','', regex=True)

### Columnas latitude y longitude
No sabemos qué información aporta y en principio no las necesitamos para nuestro análisis. Las eliminamos.

In [4]:
df = df.drop(columns=["latitude", "longitude"])

### Columna job
Quitar el ('.') al final de palabra.

In [5]:
df['job'] = df['job'].str.replace(r'\.$', '', regex=True)

### Columnas marital y poutcome
Pasar a minúsculas todas las columnas categóricas

In [6]:
spl.minus(df)

### Columna education
Cambiar ('.') por ('\_')

In [7]:
df['education'] = df['education'].str.replace('.', '_')

### Columnas nr_employed,cons.price.idx, cons.conf.idx y euribor3m
Cambiar (',') por ('.') y pasar a tipo float
- nr_employed es una variable macroeconómica (número de empleados en miles),por lo que es lógico que pase a tipo float.

In [8]:
spl.comas(df)

### Columnnas default, housing y loan
Las variables default, housing y loan representan respuestas binarias con presencia de nulos.
Para mejorar la legibilidad durante el análisis exploratorio, se han transformado a tipo categórico (object) con los valores 'Yes', 'No' y los nulos han sido reemplazados por 'unknown'.

In [9]:
df = spl.transformar_booleanos(df, columnas=['default', 'housing', 'loan'])

### Columna 'y'
La variable 'y' representa el resultado de la campaña y no presenta valores nulos.
Se ha transformado a tipo booleano para reflejar correctamente su naturaleza binaria.

In [10]:
df['y'] = df['y'].map({'yes': True, 'no': False}).astype('bool')


### Columna date
La columna `date` contiene los meses en español, lo que impide su conversión directa a tipo datetime. Se tradujeron los meses a inglés y posteriormente se transformó la columna a formato fecha. Como tiene nulos, los va a convertir a 'nat'. Posteriormente analizaremos como gestionarlos.

In [11]:
df = spl.limpiar_fecha(df, columna='date')

### Columnas contact_month, contact_year
Creamos estas dos nuevas columnnas

In [16]:
df['contact_month'] = df['date'].dt.month_name().str.lower()  # nombres de meses en inglés
df['contact_year'] = df['date'].dt.year.astype('Int64') 

In [ ]:
df.to_csv('../data/processed/01.data_limpios_bank.csv')